In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/instacomments.csv",nrows=10)

In [5]:
pd.set_option('max_colwidth',100)
data["comments"]

0                                                                        Can’t wait for season 2!!!
1                                                                                          Verified
2                                                 @harshita1210 Miss Sanyukta.. wat abt Sadda Haq??
3                                                                             Kab aayegi मिर्ज़ापुर
4                                                                                         Yayyy 🤗🤗🤗
5                                                                                        @divyenndu
6                                                                   I am Big Fan of Mirzapur Series
7    Aur kitna wait karaoge yrr tum log abhi bhabhi or raja ka Scene dekh ke kaam chalana padta hai
8                                                                                                 💞
9                                                                                               👌👌👌


In [6]:
#Convert the 'Review' Column to list for applying CountVectorizer 
Comment_list = data['comments'].values.tolist()
len('Comment_list')

12

In [7]:
#Number of Words
data['word_count'] = data['comments'].apply(lambda x: len(str(x).split(" ")))
data[['comments','word_count']].head()

,comments,word_count
0,Can’t wait for season 2!!!,5
1,Verified,1
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,7
3,Kab aayegi मिर्ज़ापुर,3
4,Yayyy 🤗🤗🤗,2


In [9]:
#Number of characters
data['char_count'] = data['comments'].str.len() ## this also includes spaces
data[['comments','char_count']].head()

,comments,char_count
0,Can’t wait for season 2!!!,26
1,Verified,8
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,49
3,Kab aayegi मिर्ज़ापुर,21
4,Yayyy 🤗🤗🤗,9


In [10]:
#Average Word Length
#Number of characters(without space count)/Total number of words
def avg_word(sentence):
  words = sentence.split()
  print(words)
  print(len(words))
  print(sum(len(word) for word in words))
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['comments'].apply(lambda x: avg_word(x))
data[['comments','avg_word']].head()

['Can’t', 'wait', 'for', 'season', '2!!!']
5
22
['Verified']
1
8
['@harshita1210', 'Miss', 'Sanyukta..', 'wat', 'abt', 'Sadda', 'Haq??']
7
43
['Kab', 'aayegi', 'मिर्ज़ापुर']
3
19
['Yayyy', '🤗🤗🤗']
2
8
['@divyenndu']
1
10
['I', 'am', 'Big', 'Fan', 'of', 'Mirzapur', 'Series']
7
25
['Aur', 'kitna', 'wait', 'karaoge', 'yrr', 'tum', 'log', 'abhi', 'bhabhi', 'or', 'raja', 'ka', 'Scene', 'dekh', 'ke', 'kaam', 'chalana', 'padta', 'hai']
19
76
['💞']
1
1
['👌👌👌']
1
3


,comments,avg_word
0,Can’t wait for season 2!!!,4.400000
1,Verified,8.000000
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,6.142857
3,Kab aayegi मिर्ज़ापुर,6.333333
4,Yayyy 🤗🤗🤗,4.000000


In [11]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['stopwords'] = data['comments'].apply(lambda x: len([x for x in x.split() if x in stop]))
data[['comments','stopwords']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,comments,stopwords
0,Can’t wait for season 2!!!,1
1,Verified,0
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,0
3,Kab aayegi मिर्ज़ापुर,0
4,Yayyy 🤗🤗🤗,0


In [14]:
#Number of special characters
data['hastags'] = data['comments'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
data[['comments','hastags']].head()

,comments,hastags
0,Can’t wait for season 2!!!,0
1,Verified,0
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,0
3,Kab aayegi मिर्ज़ापुर,0
4,Yayyy 🤗🤗🤗,0


In [15]:
#Number of numerics
data['numerics'] = data['comments'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
data[['comments','numerics']].head()

,comments,numerics
0,Can’t wait for season 2!!!,0
1,Verified,0
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,0
3,Kab aayegi मिर्ज़ापुर,0
4,Yayyy 🤗🤗🤗,0


In [16]:
#Number of Uppercase words
data['upper'] = data['comments'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
data[['comments','upper']].head()

,comments,upper
0,Can’t wait for season 2!!!,0
1,Verified,0
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,0
3,Kab aayegi मिर्ज़ापुर,0
4,Yayyy 🤗🤗🤗,0


In [17]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
from textblob import TextBlob, Word, Blobber
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
                print(ppo, tup)
    except:
        pass
    return cnt

data['noun_count'] = data['comments'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['comments'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count'] = data['comments'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count'] = data['comments'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['comments'].apply(lambda x: check_pos_tag(x, 'pron'))
data[['comments','noun_count','verb_count','adj_count', 'adv_count', 'pron_count' ]].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
NN ('wait', 'NN')
NN ('season', 'NN')
NN ('harshita1210', 'NN')
NNP ('Miss', 'NNP')
NNP ('Sanyukta..', 'NNP')
NNP ('Sadda', 'NNP')
NNP ('Haq', 'NNP')
NNP ('Kab', 'NNP')
NN ('aayegi', 'NN')
NN ('मिर्ज़ापुर', 'NN')
NNP ('Yayyy', 'NNP')
NN ('🤗🤗🤗', 'NN')
NN ('@', 'NN')
NN ('divyenndu', 'NN')
NN ('Fan', 'NN')
NNP ('Mirzapur', 'NNP')
NNP ('Series', 'NNP')
NNP ('Aur', 'NNP')
NNP ('kitna', 'NNP')
NN ('wait', 'NN')
NN ('karaoge', 'NN')
NN ('yrr', 'NN')
NN ('tum', 'NN')
NN ('log', 'NN')
NN ('bhabhi', 'NN')
NN ('raja', 'NN')
NNS ('ka', 'NNS')
NNP ('Scene', 'NNP')
NN ('ke', 'NN')
NN ('kaam', 'NN')
NN ('chalana', 'NN')
NN ('padta', 'NN')
NN ('hai', 'NN')
NN ('💞', 'NN')
NN ('👌👌👌', 'NN')
VB ('’', 'VB')
VBN ('Verified', 'VBN')
VBD ('wat', 

,comments,noun_count,verb_count,adj_count,adv_count,pron_count
0,Can’t wait for season 2!!!,2,1,1,0,0
1,Verified,0,1,0,0,0
2,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,5,1,2,0,0
3,Kab aayegi मिर्ज़ापुर,3,0,0,0,0
4,Yayyy 🤗🤗🤗,2,0,0,0,0


In [18]:
data.head()

,user name,comments,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,noun_count,verb_count,adj_count,adv_count,pron_count
0,madhura_makeupnhair,Can’t wait for season 2!!!,5,26,4.400000,1,0,0,0,2,1,1,0,0
1,battatawada\nVerified,Verified,1,8,8.000000,0,0,0,0,0,1,0,0,0
2,abby_racer20,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,7,49,6.142857,0,0,0,0,5,1,2,0,0
3,mirahul03,Kab aayegi मिर्ज़ापुर,3,21,6.333333,0,0,0,0,3,0,0,0,0
4,harshitasinha122,Yayyy 🤗🤗🤗,2,9,4.000000,0,0,0,0,2,0,0,0,0


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cv=CountVectorizer(ngram_range = (2,2), stop_words='english')
Comment_list_vec = cv.fit_transform(Comment_list)
print(Comment_list_vec.toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [21]:
# ngram range= (2,2) means only select bigrams, if we wnat both list of unigrams and bigrams we can write ngram_range=(1,2),
# token_pattern - can be changed to only use non-digit words
tv=TfidfVectorizer(ngram_range = (2,2), stop_words='english',token_pattern= r'\w+\w')
t_vec = tv.fit_transform(Comment_list)
print(t_vec.toarray())

[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.40824829 0.         0.         0.         0.
  0.         0.         0.40824829 0.         0.         0.
  0.         0.         0.         0.         0.         0.40824829
  0.         0.         0.40824829 0.40824829 0.         0.
  0.         0.         0.40824829 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         1.
  0.       

In [22]:
feature_names = tv.get_feature_names()

dense = t_vec.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
print(len(feature_names))
feature_names

28


['abhi bhabhi',
 'abt sadda',
 'aur kitna',
 'bhabhi raja',
 'big fan',
 'chalana padta',
 'dekh ke',
 'fan mirzapur',
 'harshita1210 miss',
 'ka scene',
 'kaam chalana',
 'kab aayegi',
 'karaoge yrr',
 'ke kaam',
 'kitna wait',
 'log abhi',
 'mirzapur series',
 'miss sanyukta',
 'padta hai',
 'raja ka',
 'sadda haq',
 'sanyukta wat',
 'scene dekh',
 'tum log',
 'wait karaoge',
 'wait season',
 'wat abt',
 'yrr tum']

In [23]:
df_c =pd.concat([df,data], axis=1)
df_c.head()

,abhi bhabhi,abt sadda,aur kitna,bhabhi raja,big fan,chalana padta,dekh ke,fan mirzapur,harshita1210 miss,ka scene,kaam chalana,kab aayegi,karaoge yrr,ke kaam,kitna wait,log abhi,mirzapur series,miss sanyukta,padta hai,raja ka,sadda haq,sanyukta wat,scene dekh,tum log,wait karaoge,wait season,wat abt,yrr tum,user name,comments,word_count,char_count,avg_word,stopwords,hastags,numerics,upper,noun_count,verb_count,adj_count,adv_count,pron_count
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,madhura_makeupnhair,Can’t wait for season 2!!!,5,26,4.400000,1,0,0,0,2,1,1,0,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,battatawada\nVerified,Verified,1,8,8.000000,0,0,0,0,0,1,0,0,0
2,0.0,0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.408248,0.0,0.0,0.408248,0.408248,0.0,0.0,0.0,0.0,0.408248,0.0,abby_racer20,@harshita1210 Miss Sanyukta.. wat abt Sadda Haq??,7,49,6.142857,0,0,0,0,5,1,2,0,0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,mirahul03,Kab aayegi मिर्ज़ापुर,3,21,6.333333,0,0,0,0,3,0,0,0,0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,harshitasinha122,Yayyy 🤗🤗🤗,2,9,4.000000,0,0,0,0,2,0,0,0,0
